In [19]:
#Merging all restaurant data 

import glob
import pandas as pd

# get data file names
path =r"C:/Users/YESIM/Desktop/TEXT MINING PROJECT/AMSTERDAM/RESTAURANTS/All"
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, error_bad_lines=False) )

# Concatenate all data into one DataFrame
df = pd.concat(dfs, ignore_index=True)



b'Skipping line 4: expected 5 fields, saw 6\nSkipping line 5: expected 5 fields, saw 6\nSkipping line 6: expected 5 fields, saw 6\nSkipping line 11: expected 5 fields, saw 6\nSkipping line 12: expected 5 fields, saw 6\nSkipping line 26: expected 5 fields, saw 6\nSkipping line 29: expected 5 fields, saw 6\nSkipping line 37: expected 5 fields, saw 6\nSkipping line 46: expected 5 fields, saw 6\nSkipping line 55: expected 5 fields, saw 6\nSkipping line 66: expected 5 fields, saw 6\nSkipping line 68: expected 5 fields, saw 6\nSkipping line 77: expected 5 fields, saw 6\nSkipping line 88: expected 5 fields, saw 6\nSkipping line 96: expected 5 fields, saw 6\nSkipping line 113: expected 5 fields, saw 6\nSkipping line 114: expected 5 fields, saw 6\nSkipping line 115: expected 5 fields, saw 6\nSkipping line 116: expected 5 fields, saw 6\nSkipping line 121: expected 5 fields, saw 6\nSkipping line 124: expected 5 fields, saw 6\nSkipping line 143: expected 5 fields, saw 6\nSkipping line 153: expecte

In [20]:
#Data Preparation

import numpy as np 

df = df.fillna(value=str(0))

df["full_text"] = df["title"] + ". " + df["review"]

#Reviews with 3 stars or lower are negative sentiment and others are positive sentiment 
def get_class(stars):
    score = int(stars[0])
    if score > 3:
        return 1
    else:
        return 0

def review_count(review_count):
    return review_count[0]

def help_count(help_count):
    return help_count[0]

def del_more(full_text):
    if full_text[-1] == "e" and full_text[-2] == "r" and full_text[-3] == "o" and full_text[-4] == "M":
        return full_text[0:-8]
    else:
        return full_text
    
df["review count"] = df["review_count"].apply(review_count).astype(int)
df["help count"] = df["help_count"].apply(help_count).astype(int)
df["sentiment"] = df["stars"].apply(get_class)
df["id"] = range(0,len(df))
df["full_text"] = df["full_text"].apply(del_more)

df_sent = df[["full_text", "sentiment"]]



In [21]:
#Bag of Words Method / Python Machine Learning Book

#Preprocess text data get rid of punctuations, deal with emoticons, capital letters and delete html markups by using regular expressions library
import re
def preprocess(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', '')
    
    return text
df_sent["full_text"] = df["full_text"].apply(preprocess)

#Tokenizing documents via Porter Stemmer Algorithm
def tokenizer(text):
    return text.split()
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

#Remove stop words

from nltk.corpus import stopwords
stop = stopwords.words("english")

#Splitting data into training and test sets
X = df_sent["full_text"].values
y = df_sent["sentiment"].values
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=1)

#Train Model for sentiment analysis and evaluation
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(strip_accents=None,
    lowercase=False,
    preprocessor=None, stop_words = None, ngram_range = (1,1), tokenizer = tokenizer)

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf',
    LogisticRegression(random_state=0, C=10.0, penalty = "l2"))])

lr_tfidf.fit_transform(X_train, y_train)
pred = lr_tfidf.predict(X_test)
Accuracy = lr_tfidf.score(X_test, y_test)

from sklearn.metrics import average_precision_score, recall_score

precision = average_precision_score(y_test, pred)
recall = recall_score(y_test, pred)
from sklearn.metrics import f1_score
f_score = f1_score(y_test, pred, average = "micro")

print("Accuracy: %f\nPrecision: %f\nRecall: %f\nF1 score(micro): %f" %(Accuracy, precision ,recall, f_score))
TN_Ratio = 1 - (np.sum(pred[y_test == 0])/len(pred[y_test == 0]))
print("TN/(TN+FN): %f -- Increases fundamentaly." %TN_Ratio )

C:\Users\YESIM\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\YESIM\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


Accuracy: 0.945053
Precision: 0.976670
Recall: 0.980180
F1 score(micro): 0.968095
TN/(TN+FN): 0.745242 -- Increases fundamentaly.


C:\Users\YESIM\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)


In [31]:
#Performance Evaluation on Previous Top50 Dataset

import glob
import pandas as pd

# get data file names
path =r"C:/Users/YESIM/Desktop/TEXT MINING PROJECT/AMSTERDAM/RESTAURANTS/Top 50"
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, error_bad_lines=False) )

# Concatenate all data into one DataFrame
df = pd.concat(dfs, ignore_index=True)

#Data Preparation

import numpy as np 

df = df.fillna(value=str(0))

df["full_text"] = df["title"] + ". " + df["review"]

#Reviews with 3 stars or lower are negative sentiment and others are positive sentiment 
def get_class(stars):
    score = int(stars[0])
    if score > 3:
        return 1
    else:
        return 0

def review_count(review_count):
    return review_count[0]

def help_count(help_count):
    return help_count[0]

def del_more(full_text):
    if full_text[-1] == "e" and full_text[-2] == "r" and full_text[-3] == "o" and full_text[-4] == "M":
        return full_text[0:-8]
    else:
        return full_text
    
df["review count"] = df["review_count"].apply(review_count).astype(int)
df["help count"] = df["help_count"].apply(help_count).astype(int)
df["sentiment"] = df["stars"].apply(get_class)
df["id"] = range(0,len(df))
df["full_text"] = df["full_text"].apply(del_more)

df_old = df[["full_text", "sentiment"]]

#Generating the same splits as in part 1
X_old = df_old["full_text"].values
y_old = df_old["sentiment"].values
from sklearn.cross_validation import train_test_split
X_old_train, X_old_test, y_old_train,y_old_test = train_test_split(X_old, y_old, test_size=0.33, random_state=1)

pred = lr_tfidf.predict(X_old_test)
Accuracy = lr_tfidf.score(X_old_test, y_old_test)

from sklearn.metrics import average_precision_score, recall_score

precision = average_precision_score(y_old_test, pred)
recall = recall_score(y_old_test, pred)
from sklearn.metrics import f1_score
f_score = f1_score(y_old_test, pred, average = "micro")

print("Accuracy: %f\nPrecision: %f\nRecall: %f\nF1 score(micro): %f" %(Accuracy, precision ,recall, f_score))
TN_Ratio = 1 - (np.sum(pred[y_old_test == 0])/len(pred[y_old_test == 0]))
print("TN/(TN+FN): %f -- Increases fundamentaly." %TN_Ratio )


b'Skipping line 4: expected 5 fields, saw 6\nSkipping line 5: expected 5 fields, saw 6\nSkipping line 6: expected 5 fields, saw 6\nSkipping line 11: expected 5 fields, saw 6\nSkipping line 12: expected 5 fields, saw 6\nSkipping line 26: expected 5 fields, saw 6\nSkipping line 29: expected 5 fields, saw 6\nSkipping line 37: expected 5 fields, saw 6\nSkipping line 46: expected 5 fields, saw 6\nSkipping line 55: expected 5 fields, saw 6\nSkipping line 66: expected 5 fields, saw 6\nSkipping line 68: expected 5 fields, saw 6\nSkipping line 77: expected 5 fields, saw 6\nSkipping line 88: expected 5 fields, saw 6\nSkipping line 96: expected 5 fields, saw 6\nSkipping line 113: expected 5 fields, saw 6\nSkipping line 114: expected 5 fields, saw 6\nSkipping line 115: expected 5 fields, saw 6\nSkipping line 116: expected 5 fields, saw 6\nSkipping line 121: expected 5 fields, saw 6\nSkipping line 124: expected 5 fields, saw 6\nSkipping line 143: expected 5 fields, saw 6\nSkipping line 153: expecte

Accuracy: 0.965267
Precision: 0.988630
Recall: 0.985132
F1 score(micro): 0.981619
TN/(TN+FN): 0.646018 -- Increases fundamentaly.
